# Homework 8: Deep Learning with PyTorch

## Dataset

In this homework, we'll build a model for classifying various hair types.
For this, we will use the Hair Type dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) and slightly rebuilt.

You can download the target dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip).

In [1]:
# Install specific PyTorch version as per instructions (if needed)
# !pip install torch==2.8.0 torchvision

In [2]:
import os

# Download and unzip data if not exists (Colab-ready)
if not os.path.exists("data.zip"):
    !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
    !unzip -q data.zip

--2025-11-24 11:44:42--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-24T12%3A36%3A32Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-24T11%3A36%3A27Z&ske=2025-11-24T12%3A36%3A32Z&sks=b&skv=2018-11-09&sig=nVgdGMTZf3guJ0zDuzM73EWvZ5ARm5SXQwCMPn4AvEw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2Mzk4NjQ4MiwibmJmIjoxNzYzOTg0NjgyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os
import statistics
import matplotlib.pyplot as plt
%matplotlib inline

## Reproducibility

In [4]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
# Check PyTorch version
print(f"PyTorch Version: {torch.__version__}")
# Note: The homework asks for version 2.8.0, but we will use the installed version.

PyTorch Version: 2.9.0+cu126


## Model

For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

Structure:
* Input: `(3, 200, 200)`
* Conv2d: 32 filters, kernel (3,3), relu
* MaxPool2d: (2,2)
* Flatten
* Linear: 64 neurons, relu
* Linear: 1 neuron

In [6]:
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        # Calculate input size for linear layer
        # Input: 200x200
        # Conv (3x3): 198x198
        # MaxPool (2x2): 99x99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [7]:
model = HairClassifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

Using device: cuda


### Question 1
Which loss function you will use?

In [8]:
criterion = nn.BCEWithLogitsLoss()
# We use BCEWithLogitsLoss because it combines a Sigmoid layer and the BCELoss in one single class.
# This is more numerically stable than using a plain Sigmoid followed by a BCELoss.

### Question 2
What's the total number of parameters of the model?

In [9]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


### Generators and Training

In [10]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Assuming data is unziped in current directory or 'data' directory
base_dir = '.'
if os.path.exists('data/train'):
    base_dir = 'data'
elif os.path.exists('train'):
    base_dir = '.'
else:
    # Fallback if unzipped structure is different or not found yet
    # The download cell above should handle this in Colab
    print("Warning: Data directory not found. Please ensure data is downloaded and unzipped.")

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if os.path.exists(train_dir):
    train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
    val_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

    train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
    validation_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=2)
else:
    print("Train directory not found!")

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [12]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10, start_epoch=0):
    history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

    for epoch in range(start_epoch, start_epoch + num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)

        model.eval()
        val_running_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                labels = labels.float().unsqueeze(1)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * images.size(0)
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = correct_val / total_val
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc)

        print(f"Epoch {epoch+1}, "
              f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

    return history

In [13]:
if os.path.exists(train_dir):
    history = train_model(model, train_loader, validation_loader, optimizer, criterion, num_epochs=10)

Epoch 1, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.7894, Val Acc: 0.7015


### Question 3
What is the median of training accuracy for all the epochs for this model?

In [14]:
if 'history' in locals():
    median_train_acc = statistics.median(history['acc'])
    print(f"Median training accuracy: {median_train_acc:.4f}")

Median training accuracy: 0.8175


### Question 4
What is the standard deviation of training loss for all the epochs for this model?

In [15]:
if 'history' in locals():
    std_train_loss = statistics.stdev(history['loss'])
    print(f"Standard deviation of training loss: {std_train_loss:.4f}")

Standard deviation of training loss: 0.1676


### Data Augmentation

In [16]:
train_aug_transform = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

if os.path.exists(train_dir):
    train_dataset_aug = datasets.ImageFolder(train_dir, transform=train_aug_transform)
    train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=2)

### Question 5 & 6
Train for 10 more epochs.

In [17]:
if os.path.exists(train_dir) and 'history' in locals():
    history_aug = train_model(model, train_loader_aug, validation_loader, optimizer, criterion, num_epochs=10, start_epoch=10)

Epoch 11, Loss: 0.7217, Acc: 0.6188, Val Loss: 0.5554, Val Acc: 0.7015
Epoch 12, Loss: 0.6034, Acc: 0.6850, Val Loss: 0.5599, Val Acc: 0.7214
Epoch 13, Loss: 0.5398, Acc: 0.7238, Val Loss: 0.5172, Val Acc: 0.7114
Epoch 14, Loss: 0.5217, Acc: 0.7350, Val Loss: 0.5991, Val Acc: 0.7065
Epoch 15, Loss: 0.5081, Acc: 0.7675, Val Loss: 0.5517, Val Acc: 0.7164
Epoch 16, Loss: 0.4859, Acc: 0.7575, Val Loss: 0.6278, Val Acc: 0.6866
Epoch 17, Loss: 0.4668, Acc: 0.7762, Val Loss: 0.6632, Val Acc: 0.6667
Epoch 18, Loss: 0.4860, Acc: 0.7738, Val Loss: 0.5240, Val Acc: 0.7562
Epoch 19, Loss: 0.4679, Acc: 0.7762, Val Loss: 0.6249, Val Acc: 0.6965
Epoch 20, Loss: 0.4569, Acc: 0.7812, Val Loss: 0.6747, Val Acc: 0.6766


In [18]:
if 'history_aug' in locals():
    mean_test_loss_aug = statistics.mean(history_aug['val_loss'])
    print(f"Mean of test loss (augmented training): {mean_test_loss_aug:.4f}")

Mean of test loss (augmented training): 0.5898


In [19]:
if 'history_aug' in locals():
    mean_test_acc_last5 = statistics.mean(history_aug['val_acc'][5:])
    print(f"Average test accuracy for last 5 epochs (augmented training): {mean_test_acc_last5:.4f}")

Average test accuracy for last 5 epochs (augmented training): 0.6965
